# Run CSA Algorithm
We should now have all the data needed to run the CSA algorithm. We start by loading the connections table, the footpaths tables, then by calling the CSA algorithm

In [1]:
# In this step we are loading the libriaries we will use for this part of the project
import os
import re
import pandas as pd
from IPython import get_ipython
import matplotlib.pyplot as plt
import warnings
import numpy as np
import getpass
import pyspark
import datetime as dt

pd.set_option("display.max_columns", 50)
warnings.simplefilter(action='ignore', category=UserWarning)
%matplotlib inline

In [2]:
from collections import defaultdict
from datetime import time, datetime, date
import pandas as pd

def init_algo() -> [pd.DataFrame,pd.DataFrame,pd.DataFrame,pd.DataFrame,pd.DataFrame,pd.DataFrame,list]:
    """
    Initiates the algorithm : Loads all daily connections tables, footpaths and trips list.
    """
    monday = pd.read_csv(f"../data/sbb_connections_monday_perc.csv")
    tuesday = pd.read_csv(f"../data/sbb_connections_tuesday_perc.csv")
    wednesday = pd.read_csv(f"../data/sbb_connections_wednesday_perc.csv")
    thursday = pd.read_csv(f"../data/sbb_connections_thursday_perc.csv")
    friday = pd.read_csv(f"../data/sbb_connections_friday_perc.csv")
    footpaths = pd.read_csv(f"../data/sbb_zurich_short_walks.csv")
    stops = pd.read_csv(f"../data/sbb_stations_zurich.csv")

    return monday, tuesday, wednesday, thursday, friday, footpaths, stops
monday, tuesday, wednesday, thursday, friday, footpaths, stops = init_algo()

In [3]:
import datetime as dt

def add_time(timestamp : str, duration : int) -> dt.time : 
    """
    Adds duration minutes to timestamp
    returns : addition between a time (HH:MM:SS) and a duration in minutes
    
    timestamp : a time of day (HH:MM:SS)
    duration  : minutes
    """
    delta = dt.timedelta(minutes=duration)
    dummy_date = dt.date.today()
    return (dt.datetime.combine(dummy_date, dt.datetime.strptime(timestamp, '%H:%M:%S').time()) + delta).time().strftime("%H:%M:%S")

In [4]:
def get_coords(station : str) -> list:
    """
    Gives coordinates of a station
    returns : list of ["lat","lon"]

    station : station name
    """
    global stops
    return stops.loc[stops['stop_name'] == station][["stop_lat","stop_lon"]].values.tolist()[0]

In [5]:
def recover_path(S : pd.DataFrame, starting_stop : str, end_stop : str, log) -> list:
    """
    Recovers the fastest path found by CSA algorithm.
    Algorithmically, basically iteration along a (kind of) linked list determined by starting stop and end stop.

    Returns       :  list comprised of the legs of the journey

    S             : table comprised of the latest departure time for each stop, with the associated connection
    starting_stop : start of the journey of the user
    end_stop      : end of the journey of the user
    """
    path = []
    current_leg = S[starting_stop]
    prev_trip_id = current_leg["trip_id"]
    if prev_trip_id != "Walking":
        transport_name, transport_line = current_leg["route_desc"],current_leg["route_short_name"]
    else:
        transport_name, transport_line = "Walking", ""
    prev_dept_station = current_leg["departure_station"]
    prev_dept_time = current_leg["departure_time"]
    prev_arr_time = current_leg["arrival_time"]
    prev_arr_station = current_leg["arrival_station"]
    if log:
        print(current_leg)
        
    while current_leg["trip_id"] is not None : #while we haven't arrived yet
        current_trip_id = current_leg["trip_id"]
        if current_trip_id != prev_trip_id:
            path.append([prev_trip_id, transport_name, transport_line, prev_dept_station, \
                         *get_coords(prev_dept_station), prev_dept_time, \
                         prev_arr_station, *get_coords(prev_arr_station), prev_arr_time]) 
            prev_trip_id = current_trip_id
            prev_dept_station = current_leg["departure_station"]
            prev_dept_time = current_leg["departure_time"]
            prev_arr_time = current_leg["arrival_time"]
            if prev_trip_id != "Walking":
                transport_name, transport_line = current_leg["route_desc"],current_leg["route_short_name"]
            else:
                transport_name, transport_line = "Walking", ""
        prev_arr_station = current_leg["arrival_station"]
        prev_arr_time = current_leg["arrival_time"]
        current_leg = S[current_leg["arrival_station"]] #we go to the connection of the next leg. This is like jumping to next linked list item
        
        if log:
            print(current_leg)

    path.append([prev_trip_id,transport_name, transport_line, prev_dept_station, \
                         *get_coords(prev_dept_station), prev_dept_time, \
                         prev_arr_station, *get_coords(prev_arr_station), prev_arr_time]) 
    return path

In [2]:
def CSA(starting_stop : str, arrival_stop : str, arrival_time: str, day : str, log : bool, prob : int) -> list : 
    '''
    Connections Scan Algorithm to maximize the starting time

    Return: List of connections which are each quintuples :  
            [means of transport (trip ID or 'Walking'), starting time (datetime), 
            starting_stop (name), arriving_time (datetime), arriving stop (name)]
            according to the paper

    starting_stop : Starting stop for algorithm
    arrival_stop  : End stop for algorithm
    arrival_time  : Demanded max arrival_time by the user
    day           : Day of week
    prob          : Confidence percentile (100 = delays not taken into account)
    '''
    global monday, tuesday, wednesday, thursday, friday, footpaths
    delta = 1 #minimum time to transfer in station
    percentile = {95 : "95_perc_arrival", 90 : "90_perc_arrival",85 : "85_perc_arrival", 80 : "80_perc_arrival",75 : "75_perc_arrival", 70 : "70_perc_arrival",\
                  65 : "65_perc_arrival", 60 : "60_perc_arrival",55 : "55_perc_arrival", 50 : "50_perc_arrival"}
    if prob == 100:
        arrival_index = "arrival_time"
    else:
        arrival_index = percentile[prob]
    daily_connections = {"monday" : monday, "tuesday" : tuesday, 
                         "wednesday" : wednesday, "thursday" : thursday, "friday" : friday}
    weekday_table = daily_connections[day]
    # Set all S[x] to -infty and T[x] to false
    # S : { stop : [trip_id, tentative_start_time, start_stop, arrive_time, arrive_stop]}
    S = defaultdict(lambda: pd.Series(data = [None, time.min.strftime("%H:%M:%S"), None, time.max.strftime("%H:%M:%S"), None],
                                      index = ['trip_id', 'departure_time', 'departure_station', 'arrival_time', 'arrival_station']))
    T = defaultdict(lambda : False)
    S[arrival_stop] = pd.Series(data = [None, arrival_time, None, time.max.strftime("%H:%M:%S"), None], 
                                index = ['trip_id', 'departure_time', 'departure_station', 'arrival_time', 'arrival_station'])

    # For all footpaths arriving from the arrival stop, set time as per algorithm
    for i, footpath in footpaths[footpaths["arrival"] == arrival_stop].iterrows():
        S[footpath["departure"]] = pd.Series(data = ['Walking', add_time(arrival_time,-footpath['duration']),footpath['departure'], arrival_time, arrival_stop],
                              index = ['trip_id', 'departure_time', 'departure_station', 'arrival_time', 'arrival_station'])

    # Take all connections via means of transport which arrive before wanted arrival time

    weekday_table = weekday_table[weekday_table['arrival_time']<arrival_time]
    # Core of the algorithm
    for i, connection in weekday_table.iterrows():
        # End conditition according to algorithm
        if S[starting_stop]["departure_time"] >= connection["departure_time"]:
            break 
        #If the connection is feasible, i.e. either we are on the trip that has the connection
        # or we depart after departure time + delta (min time of transfer)
        if T[connection["trip_id"]] or \
        (S[connection["arrival_station"]]["departure_time"] >= add_time(connection[arrival_index],delta)):
            # Check if the connection that allows the latest departure time is the current one
            if S[connection["departure_station"]]["departure_time"] < connection["departure_time"]:
                T[connection["trip_id"]] = True
                S[connection["departure_station"]] = connection
                # We add footpath information for nearby stations
                for _, footpath in footpaths[footpaths["arrival"] == connection["departure_station"]].iterrows():
                    if S[footpath["departure"]]["departure_time"]< add_time(connection["departure_time"],-footpath["duration"]):
                        S[footpath["departure"]] = pd.Series(data =['Walking', add_time(connection["departure_time"],-footpath["duration"]),\
                                                                    footpath['departure'], connection["departure_time"], footpath["arrival"]],
                                  index = ['trip_id', 'departure_time', 'departure_station', 'arrival_time', 'arrival_station'])
    if S[starting_stop]["trip_id"] is not None:
        return recover_path(S, starting_stop, arrival_stop, log)
    else:
        return []

In [1]:
def run(start_station : str, end_station : str, arrival_time : str, day : str, prob = 100, log = False) -> list:
    """
    Returns the shortest path in the form of a list of lists(transport, line_name, departure_station, departure_longitude, departure_latitude,
    departure_time, arrival_station, arrival_longitude, arrival_latitude, arrival_time)

    start_station : starting station of journey
    end_station   : ending station of journey
    arrival_time  : maximum arrival time
    day           : day of the week
    prob          : confidence interval (100 = delays not taken into account)
    log           : print logs of shortest path         
    """
    return CSA(start_station, end_station, arrival_time, day, log, prob)
        

# Results visualization 

This UI graphic interface allows the user to search for a desired path, specifing the arrival time and the date of the journey. 
For the sake of visualisation we use some sample journeys that will display the result of the CSA algorithm implemented in the previous part

In [8]:
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
import plotly.express as px
import plotly.graph_objects as go 
from ipywidgets import interactive, widgets, interact, BoundedIntText, Layout , Button, Label, HBox, GridBox, VBox, HTML
import datetime

## Map generating function

In [9]:
import random
def generate_map(paths : list):
    """
    Draws a map from the path output of CSA
    paths : recovered path
    """
    fig = go.Figure()
    
    trans = {"S": "S-Bahn", "B": "Bus", "T" : "Tram", "FUN" : "Funicular", "IR" : "InterRegio", "IC" : "InterCity", "BAT" : "Boat", 
             "RE" : "RegioExpress", "PB" : "Pendelbahn", "FAE" : "Ferry", "EC" : "EuroCity", "Walking": "Walking"}
    for connection in paths:
        fig.add_trace(go.Scattermapbox(
            mode="lines",
            lon=[connection[5], connection[9]],
            lat=[connection[4], connection[8]],
            line=dict(width=4, color=("#%06x" % random.randint(0, 0xFFFFFF))),
            name=trans[connection[1]]
        ))

        fig.add_trace(go.Scattermapbox(
            mode="markers",
            lon=[connection[5]],
            lat=[connection[4]],
            hoverinfo='text',
            hovertext=[connection[3]],
            showlegend=False,
            marker={'size': 15, 'color': 'black'}
        ))
        temp_lat, temp_lon, temp_name = connection[8] , connection[9], connection[7]
            
    fig.update_layout(
        title='Recommended Journey',
        title_font=dict(size=24),
        mapbox=dict(
            center=dict(
                lon=8.540212,
                lat=47.378178
            ),
            zoom=10,
            style='open-street-map'
        ),
        width=1000,
        height=600
    )

    # Add the last point marker
    last_lat = temp_lat
    last_lon = temp_lon
    fig.add_trace(go.Scattermapbox(
        mode="markers",
        lon=[last_lon],
        lat=[last_lat],
        showlegend=False,
        hoverinfo='text',
        hovertext=[temp_name],
        marker={'size': 15, 'color': 'black'}
    ))

    fig.show()

## Widget planner interface 

In [23]:
class Planner:
    def __init__(self, dep_station: str, arriv_station: str, date: datetime.date, hours: int, minutes: int, conf: int):
        self.dep_station = dep_station
        self.arriv_station = arriv_station
        self.date = date
        self.hours = hours
        self.minutes = minutes
        self.conf = conf
                
    def build(self):
        # Get stops names from previously extracted dataframe 
        stops_names = stops["stop_name"].sort_values(ascending=False).unique().tolist()
        # create widget elements for the planner 
        start_combo = widgets.Combobox(value=self.dep_station,
                                    placeholder="Choose start station",
                                    options = stops_names,
                                    description="Start : ",
                                    ensure_option=True,
                                    disabled=False)
        end_combo = widgets.Combobox(value=self.arriv_station,
                                    placeholder="Choose start station",
                                    options = stops_names,
                                    description="Start : ",
                                    ensure_option=True,
                                    disabled=False)

        prob_slider = widgets.IntSlider(
            value=100,
            min=50,
            max=self.conf,
            step=5,
            description='Confidence percentile :',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        )

        date = widgets.DatePicker(value = self.date)
        hours = BoundedIntText(value=self.hours, min=7, max=19, disabled=False, layout=Layout(width="50px"))
        minutes  = BoundedIntText(value=self.minutes, min=0, max=59, disabled=False,  layout=Layout(width="50px"))
        #search_button = Button(description='Depart', disabled=False, button_style='danger', tooltip='Click here to search for a journey')
        days_num = {0:"monday", 1:"tuesday", 2:"wednesday", 3:"thursday", 4:"friday", 5: "saturday", 6:"sunday"}
        # button action
        def display_path(path):
            s = ""
            for connection in path:
                if connection[1] in ["EC","IR","IC","S"]:
                        s+=(f"{connection[2]} from {connection[3]} at {connection[6]} -> {connection[7]} at {connection[10]} \n")
                else:
                    s+=(f"{connection[1]}{connection[2]} from {connection[3]} at {connection[6]} -> {connection[7]} at {connection[10]} \n")
            return s

        def on_button_clicked(b):
            output.clear_output() 
            
            starting_stop = start_combo.value
            arrival_stop=end_combo.value
            data = date.value
            
            day = days_num[date.value.weekday()]
            time = datetime.time(hours.value, minutes.value, 0).strftime("%H:%M:%S")
            prob = prob_slider.value
            
            with output:
                path = run(starting_stop, arrival_stop, time, day, prob)
                fig = generate_map(path)
                # fig.show()
                print("\nJourney Description:\n" + display_path(path)) 
                

        # Create the button
        search_button = widgets.Button(description='Search', button_style='danger')
        search_button.on_click(on_button_clicked)

        # stack items in a box and define the layout 
        planner_items = [
            Label(value='Departure Station'), start_combo,
            Label(value='Arrival Station'), end_combo,
            Label(value='Pick a date'), date,
            Label(value='Pick a percentile'), prob_slider,
            Label(value='Time of arrival'), HBox([hours,Label(value=":"), minutes]),
        ]

        grid = GridBox(planner_items, layout=Layout(grid_template_columns="repeat(2, 150px)"))
        return VBox([HTML("<h1> Where do you want to go? </h1>"), grid, search_button, HTML("Try 100% percentile to get the results without percentiles")], layout=Layout(padding="20px",width="650px"))


## Results Validation

### 1 - Dietlikon, Zurich Altstetten, 2023/5/31 14:00, no delay confidence

In [24]:
output = widgets.Output()
planner = Planner("Dietlikon", "Zürich Altstetten", datetime.date(2023,5,31), hours=14, minutes=0, conf=100).build()
display(planner, output)

Output()

Lets compare with Google Map's results now.

[Here we see the same journey recommend than our algorithm](https://goo.gl/maps/51U1UeCmRQSUK82V8)

For this route we have exactly the same journey recommend by Google Maps! :)

### 2 - Pfäffikon ZH, Zollikon, 2023/6/06 9:00, no delay confidence

In [26]:
output = widgets.Output()
planner = Planner("Pfäffikon ZH", "Zollikon", datetime.date(2023,6,6), hours=9, minutes=0, conf=100).build()
display(planner, output)

Output()

Lets compare with Google Map's results now. This route has buses, trains, trams and walking pathes which shows that the algorithm is indeed taking into account all means of transport.

[This is one of the recommended journey by Google Maps, like ours](https://goo.gl/maps/NkEkXFjAUoFvGBGW7)

Our journey is not the one recommend by Google Maps as this journey have too much connections, which may disturb the user. 
One addition we could make in the future to improve our algorithm and make it more user friendly, is to show the "easier" journeys first that are similar in travel time and delay probability.

### 3 - Dietlikon, Zurich Altstetten, 2023/5/31 14:00, 95% conf

In [28]:
output = widgets.Output()
planner = Planner("Dietlikon", "Zürich Altstetten", datetime.date(2023,5,31), hours=14, minutes=0, conf=95).build()
display(planner, output)

Output()

In this case running with 95% confidence intervals, we get a different route as output. 
We got this route before (no confidence):

Journey Description (no confidence):
- S19 from Dietlikon at 13:25:00 -> Wallisellen at 13:28:00 
- S14 from Wallisellen at 13:39:00 -> Zürich Oerlikon at 13:42:00 
- IR13 from Zürich Oerlikon at 13:44:00 -> Zürich HB at 13:51:00 
- IR35 from Zürich HB at 13:53:00 -> Zürich Altstetten at 13:58:00 

And at **95% confidence**, we get this new route with two stops less.

Journey Description (95% confidence):
- S19 from Dietlikon at 13:25:00 -> Wallisellen at 13:28:00 
- S14 from Wallisellen at 13:39:00 -> Zürich Altstetten at 13:54:00 

The stops ats **Oerlikon** and **Zurich HB** were eliminated from the original no confidence route. We may conclude that this is our algorithm delay computation probability altering the results, as more stations naturally leads to more delay chances.